In [1]:
import cv2 as cv

# Interactive camera viewer
Press `"b"` for blur effect

Press `"c"` for canny edge detection

Press `"t"` for simple thresholding

Press `"f"` for feature detection

Press `"a"` for ArUco marker detection

Press `"p"` for no processing

Press `"h"` for hand tracking

Press `"q"` to quit

#### Define image processing function

In [2]:
# Define processing function
def process_frame(process_function, stream:cv.VideoCapture):
    has_frame, frame = stream.read()
    if not has_frame:
        raise ConnectionError("Could not get frame from camera")
    return process_function(frame)

#### Define simple thresholding function

In [3]:
def threshold(image, threshold=125):
    _, thresh = cv.threshold(image, threshold, 255, cv.THRESH_TOZERO)
    return thresh

#### Define detect features function

In [4]:
def detect_features(frame):
    # Convert frame to grayscale
    frame_gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    
    # Detect corners
    corners = cv.goodFeaturesToTrack(frame_gray,
        maxCorners = 500,
        qualityLevel = 0.2,
        minDistance = 15,
        blockSize = 9 )
    
    if corners is not None:
        for x, y in corners[:, 0]:
            cv.circle(frame, (int(x),int(y)), 10, (0,255,0), 1)
    
    return frame
        

#### Define ArUco marker function

Use with camera pointed at `"ArUco_Marker_Board_6x6.png"` on a display/piece of paper/3D print...

In [5]:
def detect_markers(frame, aruco_dict=cv.aruco.Dictionary_get(cv.aruco.DICT_6X6_100), aruco_params=cv.aruco.DetectorParameters_create()):
    corners, ids, rejected = cv.aruco.detectMarkers(frame, aruco_dict, parameters=aruco_params)
    
    if len(corners) > 0:
        # Do marker processing here
        cv.aruco.drawDetectedMarkers(frame, corners, ids)
    
    return frame

#### Define hand tracking function

In [6]:
import mediapipe as medp
mpHands = medp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      model_complexity=0,
                      max_num_hands=2,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = medp.solutions.drawing_utils

def find_hands(frame):
    results = hands.process(frame)
    
    if results.multi_hand_landmarks:
        for handlms in results.multi_hand_landmarks:
            for id, lm in enumerate(handlms.landmark):
                #print(id, lm)
                h, w, c = frame.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                #if id == 5:
                cv.circle(frame, (cx, cy), 15, (139, 0, 0), cv.FILLED)


            mpDraw.draw_landmarks(frame, handlms, mpHands.HAND_CONNECTIONS)
    return frame

#### Define Median filtering function

In [7]:
def median_filter(frame):
    frame = cv.medianBlur(frame, 15)

    return frame

Connect to camera

In [8]:
# Connect to camera
camera_id = 1
stream = cv.VideoCapture(camera_id)

Define and run interactive image window

In [9]:
# Create Display window
window_name = f"Camera id={camera_id}"
cv.namedWindow(window_name, cv.WINDOW_NORMAL)

# Define process functions
process_functions = {
    "unprocessed" : lambda f : f,
    "blur" : lambda f : cv.blur(f, (13,13)),
    "canny" : lambda f : cv.Canny(f, 145, 150),
    "threshold" : threshold,
    "features" : detect_features,
    "aruco" : detect_markers,
    "hands" : find_hands,
    "median" : median_filter
}

# Define keybindings
class keys:
    q = ord("q")
    Q = ord("Q")
    p = ord("p")
    P = ord("P")
    b = ord("b")
    B = ord("B")
    c = ord("c")
    C = ord("C")
    t = ord("t")
    T = ord("T")
    f = ord("f")
    F = ord("F")
    a = ord("a")
    A = ord("A")
    h = ord("h")
    H = ord("H")
    m = ord("m")
    M = ord("M")
    

try:
    f = process_functions["unprocessed"]  # Initial processing function
    
    while True:
        # look for keypresses
        key = cv.waitKey(1)  # Wait 1ms
        
        # Process keypresses
        match key:
            
            case -1:  # No buttons pressed
                pass
            
            case keys.q | keys.Q | 27:  # Q or Escape
                break
                
            case keys.p | keys.P:
                f = process_functions["unprocessed"]
                
            case keys.b | keys.B:
                f = process_functions["blur"]
                
            case keys.c | keys.C:
                f = process_functions["canny"]
            
            case keys.t | keys.T:
                f = process_functions["threshold"]
            
            case keys.f | keys.F:
                f = process_functions["features"]
                
            case keys.a | keys.A:
                f = process_functions["aruco"]
            
            case keys.h | keys.H:
                f = process_functions["hands"]
            
            case keys.m | keys.M:
                f = process_functions["median"]
                
            case _:  # Unknown button pressed
                pass
            
        # Read, process, and display frame
        cv.imshow(window_name, process_frame(f, stream))

    # Close display window        
    cv.destroyWindow(window_name)
        
except Exception:
    cv.destroyWindow(window_name)

Close camera connection

In [10]:
stream.release()